<a href="https://colab.research.google.com/github/yuliiabosher/Adam_Smith_project/blob/main/Congressional_record.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import requests
import os
from google.colab import output
import pprint

In [11]:
key = input('enter API key ')
os.environ['Congress_API_Key'] = key
output.clear()

In [21]:
response = requests.get(f'https://api.congress.gov/v3/congressional-record/?y=2024&m=5&d=21&api_key={key}')
pprint.pprint(response.text)

('{\n'
 '    "Results": {\n'
 '        "IndexStart": 1,\n'
 '        "Issues": [\n'
 '            {\n'
 '                "Congress": "118",\n'
 '                "Id": 27400,\n'
 '                "Issue": "88",\n'
 '                "Links": {\n'
 '                    "Digest": {\n'
 '                        "Label": "Daily Digest",\n'
 '                        "Ordinal": 1,\n'
 '                        "PDF": [\n'
 '                            {\n'
 '                                "Part": "1",\n'
 '                                "Url": '
 '"https://www.congress.gov/118/crec/2024/05/21/170/88/CREC-2024-05-21-dailydigest.pdf"\n'
 '                            }\n'
 '                        ]\n'
 '                    },\n'
 '                    "FullRecord": {\n'
 '                        "Label": "Entire Issue",\n'
 '                        "Ordinal": 5,\n'
 '                        "PDF": [\n'
 '                            {\n'
 '                                "Part": "1",\n'
 '       

In [43]:
entire_daily_issue = response.json()['Results']['Issues'][0]['Links']['FullRecord']['PDF'][0]['Url']
print(entire_daily_issue)

https://www.congress.gov/118/crec/2024/05/21/170/88/CREC-2024-05-21.pdf
